Ignacio Vellido Expósito

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
################################################################################
# Libraries
################################################################################

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Import layers
from keras import Sequential
from keras.layers import Dense

from sklearn.model_selection import StratifiedKFold

# Import the metrics from `sklearn.metrics`
from sklearn.metrics import r2_score

################################################################################

# Basic libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
################################################################################
# Load data
################################################################################

# Read in white wine data 
white = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsc/winequality-white.csv', sep=';')

# Read in red wine data 
red = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsc/winequality-red.csv', sep=';')

# Add `type` column to `red` with value 1
red['type'] = 1

# Add `type` column to `white` with value 0
white['type'] = 0

# Append `white` to `red`
wines = red.append(white, ignore_index=True)

wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [ ]:
################################################################################
# Train-test split
################################################################################

# Specify the data
X = wines.iloc[:,0:12]
X = wines.drop('quality', axis=1) 

# Isolate target labels
#Y = wines.quality
Y = np.ravel(wines.quality)

################################################################################
# Preprocessing data
################################################################################

# Scale
X_train = StandardScaler().fit_transform(X)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python2.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
seed = 7
np.random.seed(seed)

# Parameters
splits = 5
epochs = 60

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=seed)

mse = []
mae = []
r2 = []

for train, test in kfold.split(X, Y):
  # Define model
  model = Sequential(
      [
        Dense(64, input_dim=12, activation='relu'),
        Dense(1)         
      ]
  )

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  model.fit(X_train[train], Y[train], epochs=epochs,verbose=0)


  mse_value, mae_value = model.evaluate(X_train[test], Y[test], verbose=0)
  mse.append(mse_value)
  mae.append(mae_value)

  y_pred = model.predict(X_train[test])
  r2score = r2_score(Y[test],y_pred)
  r2.append(r2score)

# Average of results in each K-fold
mse_value = np.mean(mse)
mae_value = np.mean(mae)
r2score = np.mean(r2)

print("Epochs: {}".format(epochs))
print("MSE:    {}".format(mse_value))
print("MAE:    {}".format(mae_value))
print("R2:     {}".format(r2score))

Epochs: 60
MSE:    0.482037309573
MAE:    0.538521635532
R2:     0.367790048708


Recoged los resultados anteriores en una tabla donde se indiquen los parámetros, resultados y métricas estudiados hasta el momento. Realizar estas ejecuciones para un número mayor de iteraciones del algoritmo (por ejemplo 20, 30, 40, 50…) ver qué ocurre. Comentar conclusiones

---
- Epochs: 10
- MSE:    0.524755685232
- MAE:    0.560977721214
- R2:     0.311729921596
---
- Epochs: 20
- MSE:    0.493214635154
- MAE:    0.543776381016
- R2:     0.353146244269
---
- Epochs: 30
- MSE:    0.478757350902
- MAE:    0.535899412632
- R2:     0.37209407945
---
- Epochs: 40
- MSE:    0.486252702039
- MAE:    0.543345427513
- R2:     0.362256719249
---
- Epochs: 50
- MSE:    0.479456772399
- MAE:    0.538513040543
- R2:     0.371175519533
---
- Epochs: 60
- MSE:    0.482037309573
- MAE:    0.538521635532
- R2:     0.367790048708

---

Conclusiones:
- Tenemos que fijarnos en las milésimas para observar diferencias.
Vemos que a partir de las 30 épocas comienza el sobreaprendizaje, a partir de ahí el error comienza a fluctuar.

- Respecto a la calidad en sí de los resultados, los valores de MAE nos indican que el margen de error es pequeño (equivocación de medio punto arriba/abajo en la nota otorgada). 
- A pesar de esto, los valores de R2 se ven bajos, aunque los resultados con clasificación del guiónI nos demonstraron que el problema es difícil de resolver (un tanto menor para regresión, puesto que la ausencia de datos de otras clases no penaliza de la misma forma)


In [ ]:
seed = 7
np.random.seed(seed)

# Parameters
splits = 5
epochs = 10

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=seed)

mse = []
mae = []
r2 = []

for train, test in kfold.split(X, Y):
  # Define model
  model = Sequential(
      [
        Dense(64, input_dim=12, activation='relu'),
        Dense(64, activation='relu'),
        # Dense(64, activation='relu'),
        # Dense(64, activation='relu'),
        Dense(1)         
      ]
  )

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  model.fit(X_train[train], Y[train], epochs=epochs,verbose=0)


  mse_value, mae_value = model.evaluate(X_train[test], Y[test], verbose=0)
  mse.append(mse_value)
  mae.append(mae_value)

  y_pred = model.predict(X_train[test])
  r2score = r2_score(Y[test],y_pred)
  r2.append(r2score)

# Average of results in each K-fold
mse_value = np.mean(mse)
mae_value = np.mean(mae)
r2score   = np.mean(r2)

print("Epochs: {}".format(epochs))
print("MSE:    {}".format(mse_value))
print("MAE:    {}".format(mae_value))
print("R2:     {}".format(r2score))

Epochs: 10
MSE:    0.551292880071
MAE:    0.578412055969
R2:     0.277023550101


a) Agregar más capas. Probemos a añadir más capas. Realizar la ejecución con diferentes números de capas y construir una tabla con los resultados. Mantened por el momento el mismo número de neuronas, ya que veremos el resultado de cambiar el número de las neuronas posteriormente. Construir una tabla con los parámetros y métricas utilizadas. Comentad los resultados y sacar conclusiones

---
2 capas

- Epochs: 10
- MSE:    0.551292880071
- MAE:    0.578412055969
- R2:     0.277023550101

   

- Epochs: 30
- MSE:    0.484528051438
- MAE:    0.539175724983
- R2:     0.36452036291

   

- Epochs: 50
- MSE:    0.505928785927
- MAE:    0.553491330147
- R2:     0.336542270749

    

- Epochs: 70
- MSE:    0.54414904517
- MAE:    0.564422738552
- R2:     0.286345643744

---
3 capas

- Epochs: 10
- MSE:    0.527808886028
- MAE:    0.563948392868
- R2:     0.307706186006

    

- Epochs: 30
- MSE:    0.517176227956
- MAE:    0.558221280575
- R2:     0.321760622194

   

- Epochs: 50
- MSE:    0.499126830202
- MAE:    0.540626513958
- R2:     0.345356619835

---
4 capas

- Epochs: 10
- MSE:    0.528152155888
- MAE:    0.56595979929
- R2:     0.307404602014

   

- Epochs: 30
- MSE:    0.548689993182
- MAE:    0.570972287655
- R2:     0.280310372135

   
- Epochs: 50
- MSE:    0.512767980628
- MAE:    0.546047878265
- R2:     0.327576749344

   
- Epochs: 60
- MSE:    0.55087068926
- MAE:    0.563467502594
- R2:     0.277443898965
---

Conclusiones:
- El entrenamiento correcto depende totalmente de la arquitectura.
- Modelos con más capas/unidades tienen a sobreajustar más fácilmente.

In [ ]:
seed = 7
np.random.seed(seed)

# Parameters
splits = 5
epochs = 45

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=seed)

mse = []
mae = []
r2 = []

for train, test in kfold.split(X, Y):
  # Define model
  model = Sequential(
      [
        Dense(32, input_dim=12, activation='relu'),
        # Dense(128, activation='relu'),
        # Dense(64, activation='relu'),
        # Dense(64, activation='relu'),
        Dense(1)         
      ]
  )

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  model.fit(X_train[train], Y[train], epochs=epochs,verbose=0)


  mse_value, mae_value = model.evaluate(X_train[test], Y[test], verbose=0)
  mse.append(mse_value)
  mae.append(mae_value)

  y_pred = model.predict(X_train[test])
  r2score = r2_score(Y[test],y_pred)
  r2.append(r2score)

# Average of results in each K-fold
mse_value = np.mean(mse)
mae_value = np.mean(mae)
r2score   = np.mean(r2)

print("Epochs: {}".format(epochs))
print("MSE:    {}".format(mse_value))
print("MAE:    {}".format(mae_value))
print("R2:     {}".format(r2score))

Epochs: 45
MSE:    0.488181700038
MAE:    0.541019487381
R2:     0.35970041305


Agregar más unidades ocultas. Partiendo de la configuración de una única capa y posteriormente de varias capas, probad con diferente número de neuronas. Construir una tabla que recoja tanto los parámetros utilizados, como las métricas y resultados. Comentad los resultados y sacar conclusiones.

---
1 capa, 128 unidades

- Epochs: 30
- MSE:    0.476496205197
- MAE:    0.533239984512
- R2:     0.375050078186


- Epochs: 40
- MSE:    0.481751763494
- MAE:    0.541328597069
- R2:     0.368188363539

---
2 capas, 128-64 unidades

- Epochs: 30
- MSE:    0.505831441966
- MAE:    0.549794638157
- R2:     0.336568751678
    

- Epochs: 40
- MSE:    0.496979997832
- MAE:    0.551348042488
- R2:     0.348121680966

---
2 capas, 128-128 unidades

- Epochs: 30
- MSE:    0.49993821629
- MAE:    0.545240473747
- R2:     0.344293806024

   
- Epochs: 40
- MSE:    0.495013412726
- MAE:    0.545667314529
- R2:     0.350746151389

---
1 capa, 32 unidades

- Epochs: 35
- MSE:    0.494608682757
- MAE:    0.546375501156
- R2:     0.351316947111

   
- Epochs: 40
- MSE:    0.487310701158
- MAE:    0.543989002705
- R2:     0.360886879304


- Epochs: 45
- MSE:    0.488181700038
- MAE:    0.541019487381
- R2:     0.35970041305
---
Conclusiones:
- Con una capa solo podemos modelar un hiperplano lineal, pero puede ser lo suficientenemente bueno para que la ausencia de flexibilidad en la función generalice el modelo de mejor manera.
- El aumento del número de unidades para una arquitectura que ya de por sí tiene el potencial necesario para los datos complica el ajuste correcto de la red.

---
IMPORTANTE: Recordad que hay que tener en cuenta que las redes tengan el menor número de neuronas, por varios motivos, ¿Cuáles son estos motivos?, ¿Cuándo se considera que una red sobrepasa el tamaño adecuado y deja de ser conveniente?

---
El aumento en el espacio de búsqueda puede ser innecesario para el problema en cuestión. Un aumento de capas/unidades facilita al sobreaprendizaje y ralentiza el aprendizaje, además de acarrear mayores requisitos de memoria.

Adicionalmente, muchas neuronas no permiten codificar nueva información sobre el problema, después de todo es más importante la obtención de un buen conjunto de datos.

In [ ]:
from keras.optimizers import RMSprop, SGD

seed = 7
np.random.seed(seed)

# Parameters
splits = 5
epochs = 130
lr = 1e-3

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=seed)

mse = []
mae = []
r2 = []

# opt = RMSprop(lr=lr)
opt = SGD(lr=lr)

for train, test in kfold.split(X, Y):
  # Define model
  model = Sequential(
      [
        Dense(128, input_dim=12, activation='relu'),
        # Dense(128, activation='relu'),
        # Dense(64, activation='relu'),
        # Dense(64, activation='relu'),
        Dense(1)         
      ]
  )

  model.compile(optimizer=opt, loss='mse', metrics=['mae'])
  model.fit(X_train[train], Y[train], epochs=epochs,verbose=0)


  mse_value, mae_value = model.evaluate(X_train[test], Y[test], verbose=0)
  mse.append(mse_value)
  mae.append(mae_value)

  y_pred = model.predict(X_train[test])
  r2score = r2_score(Y[test],y_pred)
  r2.append(r2score)

# Average of results in each K-fold
mse_value = np.mean(mse)
mae_value = np.mean(mae)
r2score   = np.mean(r2)

print("Epochs: {}".format(epochs))
print("LR:     {}".format(lr))
print("MSE:    {}".format(mse_value))
print("MAE:    {}".format(mae_value))
print("R2:     {}".format(r2score))

Epochs: 130
LR:     0.001
MSE:    0.480065194343
MAE:    0.538011682034
R2:     0.370371911403


Podemos probar la importación de RMSprop desde keras.models y ajustar la tasa de aprendizaje lr. También puede cambiar los valores predeterminados que se han establecido para los otros parámetros RMSprop(), pero esto no es muy recomendable


Realiza los cambios que comentamos y, además, prueba experimentar con otros algoritmos de optimización, como el Descenso de gradiente estocástico (SGD). ¿Qué efecto notas? Vuelve a recoger tus resultados en una tabla junto con los parámetros utilizados y métricas. Comenta resultados y saca conclusiones.

---
RMSProp (el usado en las pruebas anteriores era de 0.001

- Epochs: 45
- LR:     0.0001
- MSE:    0.53940390152
- MAE:    0.565697920322
- R2:     0.292503154678


- Epochs: 60
- LR:     0.0001
- MSE:    0.504027555261
- MAE:    0.54831776619
- R2:     0.338940050841


- Epochs: 80
- LR:     0.0001
- MSE:    0.488306410117
- MAE:    0.541552388668
- R2:     0.359566412447


- Epochs: 80
- LR:     0.0005
- MSE:    0.472229035967
- MAE:    0.534961915016
- R2:     0.380658607712


- Epochs: 90
- LR:     0.0005
- MSE:    0.465053232917
- MAE:    0.528471159935
- R2:     0.390073414644


- Epochs: 100
- LR:     0.0005
- MSE:    0.467848615046
- MAE:    0.532627475262
- R2:     0.386410437293
---
SGD

- Epochs: 90
- LR:     0.0005
- MSE:    0.51598752198
- MAE:    0.557888400555
- R2:     0.323246607266


- Epochs: 70
- LR:     0.0005
- MSE:    0.549419874518
- MAE:    0.572193920612
- R2:     0.279420770722


- Epochs: 30
- LR:     0.001
- MSE:    0.575189663905
- MAE:    0.58444185257
- R2:     0.245607175414


- Epochs: 60
- LR:     0.001
- MSE:    0.503936645285
- MAE:    0.551231098175
- R2:     0.3390477713


- Epochs: 90
- LR:     0.001
- MSE:    0.488905689559
- MAE:    0.542665660381
- R2:     0.358766784739


- Epochs: 110
- LR:     0.001
- MSE:    0.479930756138
- MAE:    0.538722836971
- R2:     0.370559130682


- Epochs: 130
- LR:     0.001
- MSE:    0.480065194343
- MAE:    0.538011682034
- R2:     0.370371911403
---
Conclusiones:
- SGD es más difícil de ajustar que RMSprop por tener un learning rate constante en toda la ejecución.
- Siempre se puede probar con un learning rate bajo, pues será más fácil converger dándole más épocas, pero a costa de eso existe más probabilidad de caer en un mínimo local de peor calidad.

Finalmente, De todas las que has probado, ¿Cuál sería para ti la mejor configuración del modelo que se adapta al problema? ¿Por qué? Escoger otro problema de regresión y explorar el diseño que mejor se adapte al problema

---

El mejor ha sido RMSprop con 1 capa y 128 unidades. Creemos que obtiene estos resultados porque aunque sea un perceptrón simple de una sola capa, la gran cantidad de unidades le otorga la potencia suficiente para el problema. Prevalece RMSprop sobre SGD por resultar más sencillo de ajustar apropiadamente gracias al momentum, de forma que la función de pérdida converja poco a poco con mayor facilidad a un mínimo local.

Viendo la baja diferencia en resultados, es probablemente que con una optimización de épocas con EarlyStopping muchos de los otros resultados sean capaces de obtener calidades similares.